In [1]:
import pandas as pd
pd.options.plotting.backend = "plotly"
from datetime import datetime, date
import pylatex as pl
from pylatex.utils import NoEscape
import pymc as pm
import arviz as az
import nevergrad as ng
import plotly.express as px

from estival.model import BayesianCompartmentalModel
from estival.optimization.nevergrad import optimize_model
from estival.priors import UniformPrior
from estival.targets import NegativeBinomialTarget, CustomTarget
from estival.calibration import pymc as epm
from tbdynamics import model
from tbdynamics.inputs import *
from tbdynamics.utils import build_contact_matrix
import plotly.graph_objects as go

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
time_start = 1800
time_end = 2020
time_step = 1

In [3]:
compartments = [
    "susceptible",
    "early_latent",
    "late_latent",
    "infectious",
    "on_treatment",
    "recovered",
]
infectious_compartments = [
    "infectious",
    "on_treatment",
]

latent_compartments = [
    "early_latent",
    "late_latent",
]
age_strata = [0,5,15,35,50]

In [4]:
params = {
    "contact_rate": 0.009414102898074345,
    "start_population_size": 227344.75719536067,
    "cdr_adjustment": 0.6,
    "progression_multiplier": 1.1,
    "infectious_seed": 1,
    "rr_infection_latent": 0.2,
    "rr_infection_recovered": 0.2,
    "infect_death_rate_unstratified": 0.21,
    "on_treatment_infect_multiplier": 0.08,
    'smear_positive_death_rate':0.364337776897486,
    'smear_negative_death_rate': 0.027588310343242016, 
    'smear_positive_self_recovery': 0.20344728302826143,
    'smear_negative_self_recovery': 0.22723824998716693,
    'rr_progression_diabetes': 4.5
}

In [5]:
tb_model, build_text = model.build_base_model(compartments, infectious_compartments, time_start, time_end, time_step)
build_text

"The base model consists of 6 states, representing the following states: susceptible, early_latent, late_latent, infectious, on_treatment, recovered. Only the ['infectious', 'on_treatment'] compartment contributes to the force of infection. The model is run from 1800 to 2020. "

In [6]:
model.set_starting_conditions(tb_model)


'The simulation starts with Parameter start_population_size million fully susceptible persons, with infectious persons introduced later through strain seeding as described below. '

In [7]:
model.add_entry_flow(tb_model)

'The birth process add newborns to the susceptible compartment of the model'

In [8]:
model.add_natural_death_flow(tb_model)

'The universal_death process add universal death to the model'

In [9]:
model.add_infection(tb_model)

('The infection process moves people from the susceptible compartment to the early_latent compartment, under the frequency-dependent transmission assumption. ',
 'The infection_from_latent process moves people from the late_latent compartment to the early_latent compartment, under the frequency-dependent transmission assumption. ',
 'The infection_from_recovered process moves people from the recovered compartment to the early_latent compartment, under the frequency-dependent transmission assumption. ')

In [10]:
model.add_latency(tb_model)

('The stabilisation process moves people from the early_latent compartment to the late_latent compartment, under the frequency-dependent transmission assumption. ',
 'The early_activation process moves people from the early_latent compartment to the infectious compartment, under the frequency-dependent transmission assumption. ',
 'The late_activation process moves people from the late_latent compartment to the infectious compartment, under the frequency-dependent transmission assumption. ')

In [11]:
model.add_infect_death(tb_model)

'The infect_death process moves people from the infectious'

In [12]:
model.add_self_recovery(tb_model)

'The self_recovery process moves people from the on_treatment compartment to the recovered, under the frequency-dependent transmission assumption. '

In [13]:
model.add_detection(tb_model)

'The detection process moves people from the infectious compartment to the on_treatment compartment, under the frequency-dependent transmission assumption. '

In [14]:
model.add_acf(tb_model, fixed_parameters)

[2017, 2017.01, 2018.0, 2018.01, 2019.0, 2019.01]
[0.0, 0.9750000000000001, 0.9750000000000001, 1.275, 1.275, 0.0]


'The acf_detection process moves people from the infectious compartment to the on_treatment, under the frequency-dependent transmission assumption. '

In [15]:
model.add_treatment_related_outcomes(tb_model)

('The treatment_recovery process moves people from the on_treatment compartment to the recovered compartment, under the frequency-dependent transmission assumption. ',
 'The treatment_death process moves people from the on_treatment compartment to the death, under the frequency-dependent transmission assumption. ',
 'The early_activation process moves people from the on_treatment compartment to the infectious compartment, under the frequency-dependent transmission assumption. ')

In [16]:
matrix = build_contact_matrix()

In [17]:
age_strat, des = model.add_age_strat(compartments, infectious_compartments, age_strata, matrix, fixed_parameters)
des

"The age stratification adjusts following process: (1) The universal death by age group. The data was taken from autumn's database. (2) The early and late activation(3) Age infectioness switched at age of 15(4) Infectiousness multiplier for treatment(5) Treatment outcomes: relapse, recovery and death"

In [18]:
tb_model.stratify_with(age_strat)

In [19]:
organ, des = model.add_organ_strat(fixed_parameters,infectious_compartments)
des

'The age stratification adjusts following:(1) Infectiousness adjustment by organ status(2) Different natural history (infection death) by organ status(3) Different detection rates by organ status(4) The progression rates by organ using the requested incidence proportions'

In [20]:
tb_model.stratify_with(organ)

In [21]:
gender = model.add_gender_strat(tb_model, age_strata, compartments, fixed_parameters)

In [22]:
model.request_output(tb_model, compartments,  latent_compartments, infectious_compartments)

In [23]:
priors = [
    UniformPrior("start_population_size", (150000, 300000)),
    UniformPrior("contact_rate", (0.0001, 0.02)),
    #UniformPrior("infectious_seed", [100, 2000]),
    UniformPrior("rr_infection_latent", (0.2, 0.5)),
    UniformPrior("rr_infection_recovered", (0.1, 0.5)),
    UniformPrior("smear_positive_death_rate", (0.335, 0.449)),
    UniformPrior("smear_negative_death_rate", (0.017, 0.035)),
    UniformPrior("smear_positive_self_recovery", (0.177, 0.288)),
    UniformPrior("smear_negative_self_recovery", (0.073, 0.209)),
    UniformPrior("cdr_adjustment", (0.6, 1.0)),\
    UniformPrior("rr_progression_diabetes", (2.0, 6.0))
    # UniformPrior("progression_multiplier", (0.1, 2.0)),
    # UniformPrior("cdr_adjustment", [0.6, 1.0]),
    # UniformPrior("infect_death_rate_dict.smear_positive", [0.335, 0.449]),
    # UniformPrior("infect_death_rate_dict.smear_negative", [0.017, 0.035]),
    # UniformPrior("self_recovery_rate_dict.smear_positive", [0.177, 0.288]),
    # UniformPrior("self_recovery_rate_dict.smear_negative", [0.073, 0.209]),
    # UniformPrior("rr_progression_diabetes", [1, 10]),
]
pop = pd.Series({2009: 1207100, 2019: 1194300})
notif = pd.Series({2011: 1495,2012: 1485,2013: 1369,2014:1405,2015:1642, 2016:1555, 2017:1440, 2018:1468, 2019:1417})
latent = pd.Series({2016:36})

def least_squares(modelled, obs, parameters, time_weights):
    return 0.0 - (((modelled - obs) ** 2.0)).sum()
targets = [
    CustomTarget("total_population", pop, least_squares),
   # CustomTarget("notifications", notif, least_squares),
    NegativeBinomialTarget("notifications", notif, 20.0),
    CustomTarget("percentage_latent", latent, least_squares)
]
binom_targets = [
    NegativeBinomialTarget("total_population", pop, 20.0),
]
calibration_model = BayesianCompartmentalModel(tb_model, params, priors, targets)

In [24]:
optimise_model = True
if optimise_model:
    print("Optimising with nevergrad \n Progression of loss function values:")
    optim_runner = optimize_model(calibration_model)
    for i in range(10):
        rec = optim_runner.minimize(100)
        print(rec.loss)
    optim_params = rec.value[1]
    params.update(optim_params)
    tb_model.run(parameters=params)
    print("Best calibration parameters found:")
    print(optim_params)

Optimising with nevergrad 
 Progression of loss function values:
923703556.4414456
909553258.9018126
896586181.9959801
896586181.9959801
896586181.9959801
896586181.9959801
869826912.73187
869826912.73187
869826912.73187
869826912.73187
Best calibration parameters found:
{'start_population_size': 214319.72287888144, 'contact_rate': 0.011999959566024437, 'rr_infection_latent': 0.24812096589586186, 'rr_infection_recovered': 0.28394438557193347, 'smear_positive_death_rate': 0.36720850176024517, 'smear_negative_death_rate': 0.02702614322458591, 'smear_positive_self_recovery': 0.20226403707678184, 'smear_negative_self_recovery': 0.1311606967677326, 'cdr_adjustment': 0.7298891608583179, 'rr_progression_diabetes': 3.1509872757699524}


In [33]:
tb_model.run(parameters=params)
derived_df_0 = tb_model.get_derived_outputs_df()

In [26]:
plots = {"total_population": {
      "title": "Population size",
      "output_key": "total_population",
      "times": [2009.0, 2019.0],
      "values": [1207100, 1194300],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
     "notifications": {
      "title": "Notifications",
      "output_key": "notifications",
      "times": [2011.0, 2012.0, 2013.0, 2014.0, 2015.0, 2016.0, 2017.0, 2018.0, 2019.0],
      "values": [1495, 1485, 1369, 1405, 1642, 1555, 1440, 1468, 1417],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
    "percentage_latent": {
      "title": "Percentage Latent",
      "output_key": "percentage_latent",
      "times": [2016.0],
      "values": [30.8],
      "quantiles": [0.025, 0.25, 0.5, 0.75, 0.975]
    },
    
    }

In [27]:
fig2_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="total_population",
)
fig2_2 = px.scatter(x= plots['total_population']['times'], y = plots['total_population']['values'])
fig2_2.update_traces(marker=dict(color="red"))
fig2_3 = go.Figure(
    data=fig2_1.data + fig2_2.data,
)
fig2_3.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Population"
)
fig2_3.show()

In [28]:
notif_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="notifications",
)
notif_2 = px.scatter(x= plots['notifications']['times'], y = plots['notifications']['values'])
notif_2.update_traces(marker=dict(color="red"))
notif_plot = go.Figure(
    data=notif_1.data + notif_2.data,
)
notif_plot.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Notifications"
)
notif_plot.show()

In [29]:
derived_df_0['prevalence_infectious'].plot()

In [30]:
derived_df_0['incidence'].plot()

In [31]:
latent_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="percentage_latent",
)
latent_2 = px.scatter(x= plots['percentage_latent']['times'], y = plots['percentage_latent']['values'])
latent_2.update_traces(marker=dict(color="red"))
latent_plot = go.Figure(
    data=latent_1.data + latent_2.data,
)
latent_plot.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Percentage latent"
)
latent_plot.show()

In [32]:
derived_df_0['incidence']

1800.0       0.000000
1800.1       0.501997
1800.2       1.309177
1800.3       1.817826
1800.4       2.172466
             ...     
2019.6    1135.268110
2019.7    1146.612371
2019.8    1158.636556
2019.9    1170.834206
2020.0    1182.686729
Name: incidence, Length: 2201, dtype: float64